## Calculate and save histograms of ESM4 data
To do :

- [ ] Implement basin selection

In [18]:
import xarray as xr
from xhistogram.xarray import histogram
import numpy as np
import gfdl_utils as gu
from dask.diagnostics import ProgressBar

In [68]:
config_id = 'ESM4_historical_D1'
basin = 'global'
hs_variables = ['thetao','so']
weightname = 'volcello'

bins=[]
for variable in hs_variables:
    if variable=='o2':
        binsnow = np.arange(-0.03,0.45,0.01)
    elif variable=='thetao':
        binsnow = np.arange(-2,32,0.5)
    elif variable=='so':
        binsnow = np.arange(32,38,0.1)
    bins.append(binsnow)

outdir = '../data/histograms/'
filename = 'hs.'+config_id+'.'+'-'.join(hs_variables)+'.'+weightname+'.'+basin+'.nc'

In [69]:
dmget = False

ds = xr.Dataset()
if (config_id == 'ESM4_piControl_D') or (config_id == 'ESM4_historical_D1'):

    pathDict = {'pp':('/archive/oar.gfdl.cmip6/ESM4/DECK/'+config_id+'/'+
                        'gfdl.ncrc4-intel16-prod-openmp/pp'),
                'out':'ts',
                'local':'annual/5yr',
                'time':'*'}
    pathDict['ppname']='ocean_cobalt_omip_tracers_year_z_1x1deg'
    variables = ['o2','o2sat']
    for variable in variables:
        pathDict['add'] = variable
        path=gu.core.get_pathspp(**pathDict)
        if dmget:
            gu.core.issue_dmget(path)
        else:
            ds[variable] = gu.core.open_frompp(**pathDict)[variable]
    
    pathDict['ppname']='ocean_annual_z_1x1deg'
    variables = ['thetao','so','volcello']
    for variable in variables:
        pathDict['add'] = variable
        path=gu.core.get_pathspp(**pathDict)
        if dmget:
            gu.core.issue_dmget(path)
        else:
            ds[variable] = gu.core.open_frompp(**pathDict)[variable]
    gridpath=gu.core.get_pathstatic(pathDict['pp'],pathDict['ppname'])
    if dmget:
        gu.core.issue_dmget(gridpath)
    else:
        grid = xr.open_dataset(gridpath)
    
if dmget==False:
    ds = ds.chunk({'time':1}).squeeze()

In [70]:
dslist = []
for variable in hs_variables:
    dslist.append(ds[variable])

In [71]:
if weightname == 'volcello':
    weights = ds['volcello']

hs = xr.Dataset()
hs[weightname] = histogram(*dslist,
                           bins=bins,
                           dim=['lon','lat','z_l'],
                           weights=weights)


In [72]:
with ProgressBar():
    hs.to_netcdf(outdir+filename)

[########################################] | 100% Completed | 11.0s
